In [1]:
import os
import cv2
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import segmentation_models_pytorch as smp

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
from dataset import SatelliteDataset

preprocessing_fn = smp.encoders.get_preprocessing_fn('efficientnet-b5','imagenet')

test_transform=A.Compose([
        A.Lambda(image=preprocessing_fn),
        ToTensorV2()
    ])
test_dataset = SatelliteDataset(csv_file='../data/test.csv', transform=test_transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [3]:
device="cuda" if torch.cuda.is_available() else "cpu"


In [12]:
model_path={
    "model0":"../models/ensembleUNet_model_0.pt",
    "model1":"../models/ensembleUNet_model_1.pt",
    "model2":"../models/ensembleUNet_model_2.pt",
}

models=[]

for name,path in model_path.items():
    model=smp.Unet(classes=1).to(device)
    model_state_dict=torch.load(path,map_location=device)
    model.load_state_dict(model_state_dict)
    models.append(model)
    

In [13]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [14]:
with torch.no_grad():

    for model in models:
        model.eval().to(device)

    result=[]
    for images in tqdm(test_dataloader):
        images=images.float().to(device)

        outputs_list=[]

        for model in models:
            outputs=model(images)
            outputs_list.append(outputs)

        combined_outputs=torch.mean(torch.stack(outputs_list,dim=0),dim=0)
        masks=torch.sigmoid(combined_outputs).cpu().numpy()
        masks=np.squeeze(masks,axis=1)

        masks=(masks>0.35).astype(np.uint8)

        for i in range(len(images)):
            mask_rle=rle_encode(masks[i])
            if mask_rle=='':
                result.append(-1)

            else:
                result.append(mask_rle)

    

  0%|          | 0/3790 [00:00<?, ?it/s]

100%|██████████| 3790/3790 [05:52<00:00, 10.76it/s]


In [15]:
submit = pd.read_csv('../data/sample_submission.csv')
submit['mask_rle'] = result


In [16]:
submit.to_csv('./EnsembleUNet_ep15.csv', index=False)

In [17]:
test_result=pd.read_csv('./EnsembleUNet_ep15.csv')
test_result.shape

(60640, 2)

validation set 불러오기(test)

In [15]:
new_train=pd.read_csv('../data/new_train.csv')
new_val=pd.read_csv('../data/new_val.csv')


In [23]:
new_train.head()
train_filelst=os.listdir('../data/train')
train_filelst[0]


val_filelst=os.listdir('../data/val')
val_filelst[0]

'TRAIN_1834.png'

In [19]:
cnt=0
for i in range(new_train.shape[0]):
    img_id=new_train.iloc[i][0]
    img_name=img_id+'.png'

    if img_name not in train_filelst:
        print(img_name)
        cnt+=1

print(f'csv에는 있고, 폴더에는 없는 게 {cnt}개 있음')

    

    

csv에는 있고, 폴더에는 없는 게 0개 있음


In [24]:
#validation
cnt=0
for i in range(new_val.shape[0]):
    img_id=new_val.iloc[i][0]
    img_name=img_id+'.png'

    if img_name not in val_filelst:
        print(img_name)
        cnt+=1

print(f'csv에는 있고, 폴더에는 없는 게 {cnt}개 있음')

    

csv에는 있고, 폴더에는 없는 게 0개 있음


In [22]:
cnt=0
img_list=new_train.img_id.values
img_list=[img+'.png' for img in img_list]

for img in img_list:
    if img not in train_filelst:
        print(img)
        cnt+=1

print(f'폴더에는 있고, csv에는 없는 게 {cnt}개 있음')


폴더에는 있고, csv에는 없는 게 0개 있음


In [25]:
#validation
cnt=0
img_list=new_val.img_id.values
img_list=[img+'.png' for img in img_list]

for img in img_list:
    if img not in val_filelst:
        print(img)
        cnt+=1

print(f'폴더에는 있고, csv에는 없는 게 {cnt}개 있음')

폴더에는 있고, csv에는 없는 게 0개 있음


In [26]:
new_train.head()

,img_id,img_path,mask_rle
0,TRAIN_0000,./train_img/TRAIN_0000.png,9576 7 10590 17 11614 17 12638 17 13662 17 146...
1,TRAIN_0001,./train_img/TRAIN_0001.png,208402 1 209425 6 210449 10 211473 14 212497 1...
2,TRAIN_0002,./train_img/TRAIN_0002.png,855 34 15654 9 16678 9 16742 8 17702 9 17766 9...
3,TRAIN_0005,./train_img/TRAIN_0005.png,9958 29 10982 29 12006 29 13030 29 14054 29 15...
4,TRAIN_0007,./train_img/TRAIN_0007.png,262 17 379 20 491 34 1286 17 1403 20 1515 34 2...


In [31]:
# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

class CustomDataset(Dataset):
    def __init__(self, csv_file, mode='train',transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer
        self.mode=mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        img_path=img_path.replace('./'+self.mode+'_img','/home/irteam/junghye-dcloud-dir/SpatialAI-Innovators/data/'+self.mode+'/')
    
        img_name=self.data.iloc[idx,0]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image

        mask_rle = self.data.iloc[idx, 2]
        mask = rle_decode(mask_rle, (image.shape[0], image.shape[1]))

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask,img_name

In [32]:
train_transform = A.Compose(
        [   
            A.Resize(512,512),
            A.Flip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.Lambda(image=preprocessing_fn),
            ToTensorV2()
        ]
    )

train_set=CustomDataset(csv_file='../data/new_train.csv',mode='train',transform=train_transform)

In [1]:
def save_model_state_dict(model,saved_dir,file_name='unet3+_best_model_dsv.pt'):
    output_path=os.path.join(saved_dir,file_name)
    torch.save(model.state_dict(),output_path)



In [10]:
from dataset import CustomDataset

preprocessing_fn = smp.encoders.get_preprocessing_fn('efficientnet-b5','imagenet')

train_transform = A.Compose(
        [   
            A.Resize(512,512),
            A.Flip(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.Lambda(image=preprocessing_fn),
            ToTensorV2()
        ]
    )

val_transform=A.Compose(
        [
            A.Lambda(image=preprocessing_fn),
            ToTensorV2()
        ]
    )
train_dataset = CustomDataset(csv_file='../data/new_train.csv',mode='train', transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)

val_dataset = CustomDataset(csv_file='../data/new_val.csv',mode='val', transform=val_transform)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)


  

In [12]:
for images,masks,_ in train_loader:
    print(images.shape,masks.shape)

torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])
torch.Size([4, 3, 512, 512]) torch.Size([4, 512, 512])


KeyboardInterrupt: 